# Ship Detection

## Imports

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import gc
import math
import matplotlib.pyplot as plt 
from PIL import Image
from mpl_toolkits.axes_grid1 import AxesGrid
from sklearn.metrics import fbeta_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical 
from keras import backend as K 
from keras import models, layers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
import tensorflow_probability as tfp
import tensorflow as tf
tf.config.run_functions_eagerly(True) # https://github.com/tensorflow/tensorflow/issues/34983

%matplotlib inline

## Constants

In [ ]:
ORIGINAL_IMAGE_DIMENSION = 768

## Helper Functions

In [ ]:
# Loads the metadata file with the ship segmentations and adds an extra column which indicates whether the image has any ships in it or not.
# Returns a dataframe
def load_metadata():
    df = pd.read_csv(os.path.join("..", "input", "airbus-ship-detection", "train_ship_segmentations_v2.csv"))
    df = df.groupby(["ImageId"]).agg(lambda x: x.dropna().tolist())\
        .reset_index()
    df["HasShips"] = np.where(df['EncodedPixels'].str.len() > 0, 1, 0)
    df["NumberOfShips"] = df["EncodedPixels"].str.len()
    df.set_index("ImageId", inplace=True)
    return df   

def get_preprocessed_image(df_row, dimension=ORIGINAL_IMAGE_DIMENSION, from_training_directory=True):
    img = load_image(df_row["ImageId"], from_training_directory=from_training_directory)
    img = preprocess_image(img, dimension)
    return img

# Loads an image as MxNx3 array by the imageId.
def load_image(imageId, from_training_directory = True):
    subfolder = "train_v2" if from_training_directory else "test_v2"
    imagePath = os.path.join("..", "input", "airbus-ship-detection", subfolder, imageId)
    return cv2.imread(imagePath)

# Resizing and scaling of the image
def preprocess_image(image, dimension=ORIGINAL_IMAGE_DIMENSION):
    return cv2.resize(image, (dimension, dimension)).astype('float32') / 255.

def get_mask_as_image(df_row, dimension=ORIGINAL_IMAGE_DIMENSION):
    all_masks = np.zeros((ORIGINAL_IMAGE_DIMENSION, ORIGINAL_IMAGE_DIMENSION))
    for mask in df_row["EncodedPixels"]:
        all_masks += rle_decode(mask, ORIGINAL_IMAGE_DIMENSION)
    return cv2.resize(all_masks, (dimension, dimension))

def cleanup():
    K.clear_session()
    gc.collect()

# Plots a list of images. Optionaly a list of masks can be provided
def plot_images(images, masks = None):
    cols = 4
    rows = math.ceil(len(images) / 4)
    figure = plt.figure(1, (15,20))
    grid = AxesGrid(figure, 111, nrows_ncols=(rows, cols), axes_pad=0, label_mode="1")
    
    for index, image in enumerate(images):
        grid[index].imshow(image[...,::-1]) # RGB-> BGR
        if masks is not None and masks.ndim == 3 and masks.shape[0] > index:
            grid[index].imshow(masks[index,:,:], cmap="cool", alpha=masks[index,:,:]*0.45)

def plot_model_fitting_history(history):
    plt.figure(figsize=(12,4))
    plt.subplot(1,2,(1))
    plt.plot(history.history['accuracy'],linestyle='-.')
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'valid'], loc='lower right')
    plt.subplot(1,2,(2))
    plt.plot(history.history['loss'],linestyle='-.')
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'valid'], loc='upper right')
    
# Samples the dataframe but tries to keep it balanced / stratified
def sample_and_split(df, stratify_column, test_size, sample_size):
    number_of_groups = len(df[stratify_column].unique())
    samples_per_group = int(sample_size / number_of_groups)
    balanced_df = df.groupby(stratify_column)\
        .apply(lambda x: x.sample(samples_per_group) if len(x) > samples_per_group else x)\
        .droplevel(0)\
        .reset_index()
    return train_test_split(balanced_df, test_size = test_size, stratify = balanced_df[stratify_column])

# Generic data generater that can be used to load the images in batches to avoid OOM errors and loads random samples
def random_data_generator(df, get_X, get_y, batch_size):
    X = []
    y = []
    
    while True:
        row = df.sample(n=1).iloc[0]
        X += [get_X(row)]
        y += [get_y(row)]
        if len(X) >= batch_size:
            yield np.array(X), np.array(y)
            X.clear()
            y.clear()
            
# Generic data generater that can be used to load the images in batches to avoid OOM errors
def data_generator(df, get_X, get_y, batch_size):
    X = []
    y = []
    
    while True:
        for index, row in df.iterrows():
            X += [get_X(row)]
            y += [get_y(row)]
            if len(X) >= batch_size:
                yield np.array(X), np.array(y)
                X.clear()
                y.clear()

# Run lenght encoding of an encoded mask string.
# Returns a 2 dimensional array with 0 or 1 in it
# ref: https://www.kaggle.com/paulorzp/run-length-encode-and-decode
def rle_decode(mask_rle, dimension=ORIGINAL_IMAGE_DIMENSION):
    mask = np.zeros(dimension * dimension, dtype=np.uint8)
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    
    for lo, hi in zip(starts, ends):
        mask[lo:hi] = 1
    return mask.reshape(dimension, dimension).T

## calculate intersection over union
def calculate_jaccard_coefficient_keras(y_true, y_pred, eps=1e-6):
    if np.max(y_true) == 0.0:
        return IoU(1-y_true, 1-y_pred) ## empty image; calc IoU of zeros
    intersection = K.sum(y_true * y_pred, axis=[1,2])
    union = K.sum(y_true, axis=[1,2]) + K.sum(y_pred, axis=[1,2]) - intersection
    return -K.mean( (intersection + eps) / (union + eps), axis=0)

## Exploration

In [ ]:
df_metadata = load_metadata()
df_metadata.head()

In [ ]:
print("Total number of images: " + str(df_metadata.shape[0]))
print("Images with ships:       " + str(df_metadata["HasShips"].values.sum()))
print("Minimum number of ships:" + str(df_metadata["NumberOfShips"].min()))
print("Maximum number of ships:" + str(df_metadata["NumberOfShips"].max()))

### Pie Chart

In [ ]:
ships = sum(df_metadata["HasShips"] == 1) # Number of Ships
nships = sum(df_metadata["HasShips"] == 0) # Number without Ships
total = ships + nships
ships_p = round(ships/(total)*100)
nships_p = round(nships/(total)*100)
                 
labels = "Images with ships", "Images without ships"
explode = (0, 0.1)
fig1, ax1 = plt.subplots(figsize=(15, 10))
ax1.pie([ships_p, nships_p], explode=explode, labels=labels, autopct='%1.1f%%', textprops={'fontsize': 15})
ax1.axis('equal')
plt.title('Proportion of Images with Ships', fontweight = 'bold', fontsize = 15)
plt.show()

### Bar Plot

In [ ]:
df_metadata["NumberOfShips"].plot(kind="hist", title ="Histogram of Number of Ships", figsize=(15, 10), bins=len(df_metadata["NumberOfShips"].unique()))
plt.xlabel('Number of Ships per Image', fontsize = 15)
plt.ylabel('Frequency [log]', fontsize = 15)
plt.yscale('log')
plt.title('Bar Chart with Number of Ships', fontweight = 'bold', fontsize = 15)

### RGB Histogram of every 20th Image

In [ ]:
# Lasts ca. 8 MINUTES (without GPU)
import matplotlib.ticker as ticker

nb_bins = 256
count_r = np.zeros(nb_bins)
count_g = np.zeros(nb_bins)
count_b = np.zeros(nb_bins)

for image in os.listdir('../input/airbus-ship-detection/train_v2')[::20]:
  img = Image.open('../input/airbus-ship-detection/train_v2/'+image)
  x = np.array(img)
  x = x.transpose(2, 0, 1)
  hist_r = np.histogram(x[0], bins=nb_bins, range=[0, 255])
  hist_g = np.histogram(x[1], bins=nb_bins, range=[0, 255])
  hist_b = np.histogram(x[2], bins=nb_bins, range=[0, 255])
  count_r += hist_r[0]
  count_g += hist_g[0]
  count_b += hist_b[0]

bins = hist_r[1]
fig, ax = plt.subplots(figsize=(15, 10))  
plt.bar(bins[:-1], count_r, color='r', alpha=0.7)
plt.bar(bins[:-1], count_g, color='g', alpha=0.7)
plt.bar(bins[:-1], count_b, color='b', alpha=0.7)
plt.xlabel('Pixel Brightness', fontsize = 15)
plt.ylabel('Number of Pixels', fontsize = 15)
plt.yticks(rotation=45)
plt.ticklabel_format(style = 'plain', useLocale = True) 
ax.yaxis.set_major_formatter(ticker.EngFormatter())
plt.title('RGB Histogram of every 20th Image', fontweight = 'bold', fontsize = 15)
plt.legend(['Red_Channel', 'Green_Channel', 'Blue_Channel'])

### Example Image with Mask

In [ ]:
row = df_metadata.reset_index().iloc[15]
img = get_preprocessed_image(row)
mask = get_mask_as_image(row)
plot_images([img, img], np.array([mask]))

## Simple Ship Detection

Detect whether there is a ship in the image or not

In [ ]:
# split into test and trainings data set
df_train, df_test = train_test_split(df_metadata, test_size = 0.20, stratify = df_metadata["NumberOfShips"])
# split further into trainings/validation data set

# df_train = df_train.sample(100000) # but reduce dataset size
# df_train, df_validation = train_test_split(df_train,
#                                          stratify=df_train["HasShips"],
#                                          test_size=0.25)

# split and balance trainings/validation data set
df_train, df_validation = sample_and_split(df_train,
                                          stratify_column="HasShips",
                                          test_size=0.25,
                                          sample_size=100000)

In [ ]:
SKIP_TRAINING_AND_LOAD_WEIGHTS = True

# Hyper Parameters #
BATCH_SIZE = 48
MAX_TRAINING_STEPS = 1000
CLASSES = 2
EPOCHS = 10
IMG_DIMENSION = 256
INPUT_SHAPE = (IMG_DIMENSION, IMG_DIMENSION, 3)
KERNEL_SIZE = (5, 5)
POOL_SIZE = (2, 2)
STEPS_PER_EPOCH = min(MAX_TRAINING_STEPS, df_train.shape[0]//BATCH_SIZE)
VALIDATION_STEPS = 100


trainings_generator = data_generator(df_train.reset_index(), 
                                  get_X= lambda row: np.array(get_preprocessed_image(row, IMG_DIMENSION)),
                                  get_y= lambda row: to_categorical(row["HasShips"], CLASSES),
                                  batch_size = BATCH_SIZE)
validation_generator = random_data_generator(df_validation.reset_index(), 
                                  get_X= lambda row: np.array(get_preprocessed_image(row, IMG_DIMENSION)),
                                  get_y= lambda row: to_categorical(row["HasShips"],  CLASSES),
                                  batch_size = BATCH_SIZE)

# create model
model = models.Sequential()

model.add(layers.Convolution2D(8,KERNEL_SIZE,padding='same',input_shape=INPUT_SHAPE))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Convolution2D(8, KERNEL_SIZE,padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=POOL_SIZE))

model.add(layers.Convolution2D(16, KERNEL_SIZE,padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))

model.add(layers.Convolution2D(16,KERNEL_SIZE,padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=POOL_SIZE))

model.add(layers.Flatten())
model.add(layers.Dense(40))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))
model.add(layers.Activation('relu'))
model.add(layers.Dense(CLASSES))
model.add(layers.Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

if SKIP_TRAINING_AND_LOAD_WEIGHTS:
    model.load_weights(os.path.join("..", "input", "airbusshipdetectionmodels", "simple_model_10e_sampled.h5"))
else:
    # train the model
    history=model.fit(trainings_generator, 
                      validation_data=validation_generator,
                      epochs=EPOCHS,
                      steps_per_epoch=STEPS_PER_EPOCH,
                      validation_steps=VALIDATION_STEPS,
                      verbose=1)
    model.save_weights("simple_model.h5")
    plot_model_fitting_history(history)

### Evaluation with Test Set

In [ ]:
test_generator = random_data_generator(df_test.reset_index(), 
                                      get_X= lambda row: np.array(get_preprocessed_image(row, IMG_DIMENSION)),
                                      get_y= lambda row: to_categorical(row["HasShips"], CLASSES),
                                      batch_size = 1000)

# predict random 1000 samples (the whole test data set takes too much time)
X_test, y_test = next(test_generator)
y_pred = model.predict(X_test)

In [ ]:
print('\n', classification_report(np.where(y_test > 0)[1], 
np.argmax(y_pred, axis=1)), sep='')                                        

In [ ]:
plt.figure(figsize=(8,8))
cnf_matrix = confusion_matrix(np.where(y_test > 0)[1], np.argmax(y_pred, axis=1))
class_labels = list(["no-ship", "ship"])
plt.imshow(cnf_matrix, interpolation='nearest')
plt.colorbar()
tick_marks = np.arange(CLASSES)
_ = plt.xticks(tick_marks, class_labels, rotation=90)
_ = plt.yticks(tick_marks, class_labels)

## Multinomial Classifcation
Detect wheter there is no ship, one, two or more ships in the image


In [ ]:
df_metadata["NumerOfShips0to3"]= df_metadata.apply(lambda x: 3 if x["NumberOfShips"] >= 3 else x["NumberOfShips"], axis=1)  
df_metadata["NumerOfShips0to3"].hist()

In [ ]:
SKIP_TRAINING_AND_LOAD_WEIGHTS = True

# here we define  hyperparameter of the NN
batch_size = 48
max_training_steps = 1000
classes = 4
epochs = 10
img_dimension = 256
kernel_size = (5, 5)
input_shape = (img_dimension, img_dimension, 3)
pool_size = (2, 2)

# split into test and trainings data set
df_train, df_test = train_test_split(df_metadata, test_size = 0.25, stratify = df_metadata["NumerOfShips0to3"])
# split and balance trainings/validation data set
df_train, df_validation = sample_and_split(df_train,
                                          stratify_column="NumerOfShips0to3",
                                          test_size=1/3,
                                          sample_size=40000)


steps_per_epoch = min(max_training_steps, df_train.shape[0]//batch_size)
trainings_generator = data_generator(df_train.reset_index(), 
                                  get_X= lambda row: np.array(get_preprocessed_image(row, img_dimension)),
                                  get_y= lambda row: to_categorical(row["NumerOfShips0to3"], classes),
                                  batch_size = batch_size)
validation_generator = random_data_generator(df_validation.reset_index(), 
                                  get_X= lambda row: np.array(get_preprocessed_image(row, img_dimension)),
                                  get_y= lambda row: to_categorical(row["NumerOfShips0to3"], classes),
                                  batch_size = batch_size)

# create model
model = models.Sequential()

#model.add(BatchNormalization(input_shape=input_shape))
model.add(layers.Convolution2D(8,kernel_size,padding='same',input_shape=input_shape))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Convolution2D(8, kernel_size,padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=pool_size))

model.add(layers.Convolution2D(16, kernel_size,padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Convolution2D(16,kernel_size,padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=pool_size))
          
model.add(layers.Convolution2D(64, kernel_size,padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Convolution2D(64,kernel_size,padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=pool_size))          

model.add(layers.Flatten())
model.add(layers.Dense(500))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(300))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(100))
model.add(layers.Activation('relu'))

model.add(layers.Dense(classes))
model.add(layers.Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

if SKIP_TRAINING_AND_LOAD_WEIGHTS:
    model.load_weights(os.path.join("..", "input", "airbusshipdetectionmodels", "multi_model_60.h5"))
else:
    # train the model
    history=model.fit(trainings_generator, 
                      validation_data=validation_generator,
                      epochs=epochs,
                      steps_per_epoch=steps_per_epoch,
                      validation_steps=100,
                      verbose=1)
    model.save_weights("multi_model.h5")
    plot_model_fitting_history(history)

In [ ]:
test_generator = random_data_generator(df_test.reset_index(), 
                                      get_X= lambda row: np.array(get_preprocessed_image(row, img_dimension)), 
                                      get_y= lambda row: to_categorical(row["NumerOfShips0to3"], classes),
                                      batch_size = 1000)

# predict 1000 samples
X_test, y_test = next(test_generator)
y_pred = model.predict(X_test)

In [ ]:
print('\n', classification_report(np.where(y_test > 0)[1], 
                                  np.argmax(y_pred, axis=1)), sep='')     

In [ ]:
plt.figure(figsize=(8,8))
cnf_matrix = confusion_matrix(np.where(y_test > 0)[1], np.argmax(y_pred, axis=1), normalize='true')
class_labels = list(["no-ship", "1 ship", "2 ship", ">=3 ship"])
plt.imshow(cnf_matrix, interpolation='nearest')
plt.colorbar()
tick_marks = np.arange(classes)
_ = plt.xticks(tick_marks, class_labels, rotation=90)
_ = plt.yticks(tick_marks, class_labels)

## Gaussian CPD
Counting ships using Gaussian CPD (flexible mean & variance)

### !!!Continue with "Image Segmentation" or close the session and restart it with Gaussian CPD!!!

### Install old Version
Versions like 2.4.1 and 0.8.0 are not able to calculate measures of position like "mean" and "standard deviation".

In [ ]:
# Make sure you enabled "internet connection" on the right hand side
!pip install tensorflow==2.1.0

In [ ]:
!pip install tensorflow_probability==0.8.0

### Imports

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import gc
import math
import matplotlib.pyplot as plt 
from mpl_toolkits.axes_grid1 import AxesGrid
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical 

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Convolution2D, MaxPooling2D, Flatten , Activation, Dropout, Input, Concatenate
import tensorflow_probability as tfp
tfd = tfp.distributions

from tensorflow.keras.models import Sequential

import tensorflow as tf

%matplotlib inline

### Check Version

In [ ]:
print("TF  Version",tf.__version__)
print("TFP Version", tfp.__version__)

### Constants

In [ ]:
ORIGINAL_IMAGE_DIMENSION = 768

### Helper Functions

In [ ]:
# Loads the metadata file with the ship segmentations and adds an extra column which indicates whether the image has any ships in it or not.
# Returns a dataframe
def load_metadata():
    df = pd.read_csv(os.path.join("..", "input", "airbus-ship-detection", "train_ship_segmentations_v2.csv"))
    df = df.groupby(["ImageId"]).agg(lambda x: x.dropna().tolist())\
        .reset_index()
    df["HasShips"] = np.where(df['EncodedPixels'].str.len() > 0, 1, 0)
    df["NumberOfShips"] = df["EncodedPixels"].str.len()
    df.set_index("ImageId", inplace=True)
    return df   

def get_preprocessed_image(df_row, dimension=ORIGINAL_IMAGE_DIMENSION):
    img = load_image(df_row["ImageId"], from_training_directory=True)
    img = preprocess_image(img, dimension)
    return img

# Loads an image as MxNx3 array by the imageId.
def load_image(imageId, from_training_directory = True):
    subfolder = "train_v2" if from_training_directory else "test_v2"
    imagePath = os.path.join("..", "input", "airbus-ship-detection", subfolder, imageId)
    return cv2.imread(imagePath)

# Resizing and scaling of the image
def preprocess_image(image, dimension=ORIGINAL_IMAGE_DIMENSION):
    return cv2.resize(image, (dimension, dimension)).astype('float32') / 255.

def get_mask_as_image(df_row, dimension=ORIGINAL_IMAGE_DIMENSION):
    all_masks = np.zeros((ORIGINAL_IMAGE_DIMENSION, ORIGINAL_IMAGE_DIMENSION))
    for mask in df_row["EncodedPixels"]:
        all_masks += rle_decode(mask)
    return cv2.resize(all_masks, (dimension, dimension))

def cleanup():
    K.clear_session()
    gc.collect()

# Plots a list of images. Optionaly a list of masks can be provided
def plot_images(images, masks = []):
    cols = 4
    rows = math.ceil(len(images) / 4)
    figure = plt.figure(1, (15,20))
    grid = AxesGrid(figure, 111, nrows_ncols=(rows, cols), axes_pad=0, label_mode="1")
    
    for index, image in enumerate(images):
        grid[index].imshow(image[...,::-1]) # RGB-> BGR
        if len(masks) > index:
            grid[index].imshow(masks[index], cmap="cool", alpha=mask*0.65)

def plot_model_fitting_history(history):
    plt.figure(figsize=(12,4))
    plt.subplot(1,2,(1))
    plt.plot(history.history['accuracy'],linestyle='-.')
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'valid'], loc='lower right')
    plt.subplot(1,2,(2))
    plt.plot(history.history['loss'],linestyle='-.')
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'valid'], loc='upper right')
    
# Samples the dataframe but tries to keep it balanced / stratified
def sample_and_split(df, stratify_column, test_size, sample_size):
    number_of_groups = len(df[stratify_column].unique())
    samples_per_group = int(sample_size / number_of_groups)
    balanced_df = df.groupby(stratify_column)\
        .apply(lambda x: x.sample(samples_per_group) if len(x) > samples_per_group else x)\
        .droplevel(0)\
        .reset_index()
    return train_test_split(balanced_df, test_size = test_size, stratify = balanced_df[stratify_column])

# Generic data generater that can be used to load the images in batches to avoid OOM errors and loads random samples
def random_data_generator(df, get_X, get_y, batch_size):
    X = []
    y = []
    
    while True:
        row = df.sample(n=1).iloc[0]
        X += [get_X(row)]
        y += [get_y(row)]
        if len(X) >= batch_size:
            yield np.array(X), np.array(y)
            X.clear()
            y.clear()
            
# Generic data generater that can be used to load the images in batches to avoid OOM errors
def data_generator(df, get_X, get_y, batch_size):
    X = []
    y = []
    
    while True:
        for index, row in df.iterrows():
            X += [get_X(row)]
            y += [get_y(row)]
            if len(X) >= batch_size:
                yield np.array(X), np.array(y)
                X.clear()
                y.clear()

# Run lenght encoding of an encoded mask string.
# Returns a 2 dimensional array with 0 or 1 in it
def rle_decode(mask_rle):
    mask = np.zeros(ORIGINAL_IMAGE_DIMENSION * ORIGINAL_IMAGE_DIMENSION, dtype=np.uint8)
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    
    for lo, hi in zip(starts, ends):
        mask[lo:hi] = 1
    return mask.reshape(ORIGINAL_IMAGE_DIMENSION, ORIGINAL_IMAGE_DIMENSION).T

### Metadata

In [ ]:
df_metadata = load_metadata()
df_metadata.head()

### Preparation

In [ ]:
df_metadata= df_metadata.loc[df_metadata["NumberOfShips"] > 0] # at least one ship

# split into test and trainings data set
df_train, df_test = train_test_split(df_metadata, test_size = 0.20, stratify = df_metadata["NumberOfShips"])
# split further into trainings/validation data set

# df_train = df_train.sample(100000) # but reduce dataset size
# df_train, df_validation = train_test_split(df_train,
#                                          stratify=df_train["HasShips"],
#                                          test_size=0.25)

# split and balance trainings/validation data set
df_train, df_validation = sample_and_split(df_train,
                                          stratify_column="NumberOfShips",
                                          test_size=0.25,
                                          sample_size=100000)

In [ ]:
plt.figure(figsize=(14,6))
plt.subplot(1,2,1)
plt.hist(df_train["NumberOfShips"],bins=30)
plt.title("Number of Ships: train")
plt.subplot(1,2,2)
plt.hist(df_validation["NumberOfShips"],bins=30)
plt.title("Number of Ships: val")

### Training

In [ ]:
SKIP_TRAINING_AND_LOAD_WEIGHTS = True

# here we define hyperparameter of the NN
BATCH_SIZE = 48
MAX_TRAINING_STEPS = 1000
EPOCHS = 15
IMG_DIMENSION = 256
INPUT_SHAPE = (IMG_DIMENSION, IMG_DIMENSION, 3)
KERNEL_SIZE = (3, 3)
POOL_SIZE = (2, 2)
STEPS_PER_EPOCH = min(MAX_TRAINING_STEPS, df_train.shape[0]//BATCH_SIZE)
VALIDATION_STEPS = 100

# traininigs- /validation_generator
trainings_generator = data_generator(df_train.reset_index(), 
                                  get_X= lambda row: np.array(get_preprocessed_image(row, IMG_DIMENSION)),
                                  get_y= lambda row: np.asarray(row["NumberOfShips"]).astype("float32"),
                                  batch_size = BATCH_SIZE)
validation_generator = random_data_generator(df_validation.reset_index(), 
                                  get_X= lambda row: np.array(get_preprocessed_image(row, IMG_DIMENSION)),
                                  get_y= lambda row: np.asarray(row["NumberOfShips"]).astype("float32"),
                                  batch_size = BATCH_SIZE)


# model
def NLL(y, distr):
  return -distr.log_prob(y) 

def my_dist(params): 
  return tfd.Normal(loc=params[:,0:1], scale=1e-3 + tf.math.softplus(0.05 * params[:,1:2]))# both parameters are learnable

inputs = Input(shape=(INPUT_SHAPE))
x = Convolution2D(8,KERNEL_SIZE,padding='same',activation="relu")(inputs)
x = Convolution2D(8,KERNEL_SIZE,padding='same',activation="relu")(x)
x = MaxPooling2D(pool_size=POOL_SIZE)(x)

x = Convolution2D(16,KERNEL_SIZE,padding='same',activation="relu")(x)
x = Convolution2D(16,KERNEL_SIZE,padding='same',activation="relu")(x)
x = MaxPooling2D(pool_size=POOL_SIZE)(x)

x = Flatten()(x)
x = Dropout(0.3)(x)
x = Dense(256,activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(64,activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(2)(x)
dist = tfp.layers.DistributionLambda(my_dist)(x) 

model_flex = Model(inputs=inputs, outputs=dist)
model_flex.compile(tf.keras.optimizers.Adam(), loss=NLL)

if SKIP_TRAINING_AND_LOAD_WEIGHTS:
    model_flex.load_weights(os.path.join("..", "input", "airbusshipdetectionmodels", "adv_model2.h5"))
else:
    # train the model
    history=model_flex.fit(trainings_generator, 
                           validation_data=validation_generator,
                           epochs=EPOCHS,
                           steps_per_epoch=STEPS_PER_EPOCH,
                           validation_steps=VALIDATION_STEPS,
                           verbose=1)
    model_flex.save_weights("adv_model2.h5")

### Evaluation

mean, standard deviation

In [ ]:
model_mean = Model(inputs=inputs, outputs=dist.mean())
model_sd = Model(inputs=inputs, outputs=dist.stddev())

Predict 1000 samples

In [ ]:
test_generator = random_data_generator(df_test.reset_index(), 
                                       get_X= lambda row: np.array(get_preprocessed_image(row, IMG_DIMENSION)),
                                       get_y= lambda row: np.asarray(row["NumberOfShips"]).astype("float32"),
                                       batch_size = 1000)

# predict 1000 samples
X_test, y_test = next(trainings_generator)
y_pred = model_flex.predict(X_test)

Look at the predicted mean of the CPD on the testset

In [ ]:
plt.figure(figsize=(20,20))
for i in range(0,25):
    plt.subplot(5,5,i+1)
    plt.imshow(X_test[i])
    plt.title("pred : "+ np.str(model_mean.predict(X_test[i:i+1])[0][0]) + "   true : "+ np.str(y_test[i]))

Look at the predicted mean and the predicted sigma of the CPD on the testset

In [ ]:
for i in range(0,10):
  plt.figure(figsize=(12,6))
  plt.subplot(1,2,1)
  plt.imshow(X_test[i])
  plt.title("pred : "+ np.str(model_mean.predict(X_test[i:i+1])[0][0]) + "   true : "+ np.str(y_test[i]))
  print(model_mean.predict(X_test[i:i+1]))
  print(model_sd.predict(X_test[i:i+1]))
  d = tfd.Normal(loc=model_mean.predict(X_test[i:i+1]), scale=model_sd.predict(X_test[i:i+1]))           #A
  plt.subplot(1,2,2)
  plt.plot(np.arange(-10,100,0.2),d.prob(np.arange(-10,100,0.2))[0])
  plt.show()

Indicator

In [ ]:
mse=np.average(np.square(model_mean.predict(X_test).flatten()-y_test))
rmse=np.sqrt(mse)
nll=model_flex.evaluate(X_test,y_test,verbose=0)
df= pd.DataFrame(
         { 'MSE' : mse, 'RMSE' : rmse, 'nll ' : nll}, index=['model flex sigma'])
df

### Reload Metadata


In [ ]:
df_metadata = load_metadata()
df_metadata.head()

## Image Segmentation

Detect all the ships in an image with their bounding boxes

In [ ]:
### split into test and trainings data set
df_train, df_test = train_test_split(df_metadata, test_size = 0.25, stratify = df_metadata["NumberOfShips"])
# split and balance trainings/validation data set
df_train, df_validation = sample_and_split(df_metadata,
                                          stratify_column="NumberOfShips",
                                          test_size=0.5,
                                          sample_size=10000)

In [ ]:
SKIP_TRAINING_AND_LOAD_WEIGHTS = True
WEIGHT_CHECKPOINT_PATH = "{}_weights.best.hdf5".format('seg_model')

# Hyper Parameters #
BATCH_SIZE = 24
GAUSSIAN_NOISE = 0.1
NET_SCALING = (1, 1) # optional downsampling inside the network (disabled)
MAX_TRAINING_STEPS = 9
MAX_TRAIN_EPOCHS = 99
IMG_DIMENSION = 256
INPUT_SHAPE = (IMG_DIMENSION, IMG_DIMENSION, 3)
STEPS_PER_EPOCH = min(MAX_TRAINING_STEPS, df_train.shape[0]//BATCH_SIZE)
VALIDATION_STEPS = 100

trainings_generator = data_generator(df_train.reset_index(), 
                                  get_X= lambda row: np.array(get_preprocessed_image(row, dimension=IMG_DIMENSION)),
                                  get_y= lambda row: np.array(get_mask_as_image(row, dimension=IMG_DIMENSION)),
                                  batch_size = BATCH_SIZE)
validation_generator = random_data_generator(df_validation.reset_index(), 
                                  get_X= lambda row: np.array(get_preprocessed_image(row, dimension=IMG_DIMENSION)),
                                  get_y= lambda row: np.array(get_mask_as_image(row, dimension=IMG_DIMENSION)),
                                  batch_size = BATCH_SIZE)

# create model
input_img = layers.Input(INPUT_SHAPE, name = 'RGB_Input')

pp_in_layer = input_img
pp_in_layer = layers.AvgPool2D(NET_SCALING)(pp_in_layer)
pp_in_layer = layers.GaussianNoise(GAUSSIAN_NOISE)(pp_in_layer)
pp_in_layer = layers.BatchNormalization()(pp_in_layer)

c1 = layers.Conv2D(8, (3, 3), activation='relu', padding='same') (pp_in_layer)
c1 = layers.Conv2D(8, (3, 3), activation='relu', padding='same') (c1)
p1 = layers.MaxPooling2D((2, 2)) (c1)

c2 = layers.Conv2D(16, (3, 3), activation='relu', padding='same') (p1)
c2 = layers.Conv2D(16, (3, 3), activation='relu', padding='same') (c2)
p2 = layers.MaxPooling2D((2, 2)) (c2)

c3 = layers.Conv2D(32, (3, 3), activation='relu', padding='same') (p2)
c3 = layers.Conv2D(32, (3, 3), activation='relu', padding='same') (c3)
p3 = layers.MaxPooling2D((2, 2)) (c3)

c4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same') (p3)
c4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same') (c4)
p4 = layers.MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = layers.Conv2D(128, (3, 3), activation='relu', padding='same') (p4)
c5 = layers.Conv2D(128, (3, 3), activation='relu', padding='same') (c5)

u6 = layers.UpSampling2D((2, 2)) (c5)
u6 = layers.concatenate([u6, c4])
c6 = layers.Conv2D(64, (3, 3), activation='relu', padding='same') (u6)
c6 = layers.Conv2D(64, (3, 3), activation='relu', padding='same') (c6)

u7 = layers.UpSampling2D((2, 2)) (c6)
u7 = layers.concatenate([u7, c3])
c7 = layers.Conv2D(32, (3, 3), activation='relu', padding='same') (u7)
c7 = layers.Conv2D(32, (3, 3), activation='relu', padding='same') (c7)

u8 = layers.UpSampling2D((2, 2)) (c7)
u8 = layers.concatenate([u8, c2])
c8 = layers.Conv2D(16, (3, 3), activation='relu', padding='same') (u8)
c8 = layers.Conv2D(16, (3, 3), activation='relu', padding='same') (c8)

u9 = layers.UpSampling2D((2, 2)) (c8)
u9 = layers.concatenate([u9, c1], axis=3)
c9 = layers.Conv2D(8, (3, 3), activation='relu', padding='same') (u9)
c9 = layers.Conv2D(8, (3, 3), activation='relu', padding='same') (c9)

d = layers.Conv2D(1, (1, 1), activation='sigmoid') (c9)
d = layers.UpSampling2D(NET_SCALING)(d)

seg_model = models.Model(inputs=[input_img], outputs=[d])

#seg_model.summary()
tf.keras.utils.plot_model(seg_model)

In [ ]:
callbacks_list = [ModelCheckpoint(WEIGHT_CHECKPOINT_PATH, monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True),
                  EarlyStopping(monitor="val_loss", mode="min", verbose=2, patience=20), # probably needs to be more patient, but kaggle time is limited
                  ReduceLROnPlateau(monitor='val_loss', factor=0.33,
                                    patience=1, verbose=1, mode='min',
                                    min_delta=0.0001, cooldown=0, min_lr=1e-8)]

In [ ]:
def fit():
    seg_model.compile(optimizer=Adam(1e-3, decay=1e-6), 
                      loss=calculate_jaccard_coefficient_keras, 
                      metrics=['binary_accuracy'])
    loss_history = [seg_model.fit(trainings_generator,
                                  validation_data = validation_generator,
                                  validation_steps = STEPS_PER_EPOCH//2,
                                  epochs = MAX_TRAIN_EPOCHS,
                                  steps_per_epoch = STEPS_PER_EPOCH,
                                  callbacks = callbacks_list, 
                                  workers = 1,
                                  verbose = 1)]
    
    return loss_history

if (SKIP_TRAINING_AND_LOAD_WEIGHTS):
    seg_model.load_weights(os.path.join("..", "input", "airbusshipdetectionmodels", "segmentation_model.h5"))
else:
    while True:
        loss_history = fit()
        if np.min([mh.history['val_loss'] for mh in loss_history]) < -0.2:
            break

    plot_model_fitting_history(history)

### Visualize Prediction & Evaluation

In [ ]:
test_generator = random_data_generator(df_test.reset_index(), 
                                  get_X= lambda row: np.array(get_preprocessed_image(row, dimension=IMG_DIMENSION)),
                                  get_y= lambda row: np.array(get_mask_as_image(row, dimension=IMG_DIMENSION)),
                                  batch_size = 1000)

X_test, y_test = next(test_generator)
y_pred_seg = seg_model.predict(X_test)
y_pred_bin = model.predict(X_test)

In [ ]:
plot_images(X_test[0:24], np.array(y_test[0:24]))

In [ ]:
plot_images(X_test[0:24], np.array(y_pred_seg[0:24,:,:,0]))

In [ ]:
i = 12 # Compare just one image
plot_images([X_test[i],X_test[i]], np.array([y_test[i], y_pred_seg[i,:,:,0]]))

### Scoring

In [ ]:
def score(y_test, y_pred):
    pred = y_pred[:,:,:,0] if y_pred.ndim == 4 else y_pred
    pred = (pred > 0.5).astype(float)
    score = 0

    for i in range(y_test.shape[0]):
        score += fbeta_score(y_test[i].flatten(), pred[i].flatten(), beta=2, zero_division=1)
    return score / y_test.shape[0]

def ensemble_score(y_test, y_pred_seg, y_pred_bin):
    y_pred = []
    y_pred_bin = np.argmax(y_pred_bin, axis=1)
    for i in range(y_pred_seg.shape[0]):
        if y_pred_bin[i] == 1:
            y_pred += [y_pred_seg[i]]
        else:
            y_pred += [np.zeros((y_pred_seg.shape[1], y_pred_seg.shape[2], 1))]
    return score(y_test, np.array(y_pred))

In [ ]:
print("FBeta Score: ", score(y_test, y_pred_seg))
print("FBeta Score with Ensemble:", ensemble_score(y_test, y_pred_seg, y_pred_bin))